In [1]:
import pandas as pd

In [2]:
data=pd.read_csv("../Dataset/CSV-Single/10_finale.csv",index_col="Reg_No")

In [3]:
data.drop(['S1_F','S1_F_mark','S1_F_att','S1_F_grade'],axis=1,inplace=True)
data.dropna(inplace=True)

In [4]:
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
import re
from sklearn.base import BaseEstimator, TransformerMixin

class DivideByHundred(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        return X/100
    def get_feature_names_out(self, features=None):
        return features

In [5]:
ohe=OneHotEncoder(sparse_output=False)
oe=OrdinalEncoder(categories=[['F','P','D','C','C+','B','B+','A','A+','S']]*len([x for x in data.columns if re.fullmatch('^S[12]_[A-FST]_grade$',x)]))
dbh=DivideByHundred()
ct=ColumnTransformer([
    ("scheme",ohe,['Scheme']),
    ('dept',ohe,['Dept']),
    ('sub',ohe,[x for x in data.columns if re.fullmatch('^S[12]_[A-FST]$',x)]),
    ('mark_and_att',dbh,[x for x in data.columns if re.fullmatch('^S[12]_[A-FST]_(?:att|mark)$',x)]),
    ('grade',oe,[x for x in data.columns if re.fullmatch('^S[12]_[A-FST]_grade$',x)])
],n_jobs=-1,verbose=True,verbose_feature_names_out=False)

In [6]:
encoded_data=pd.DataFrame(ct.fit_transform(data),index=data.index,columns=ct.get_feature_names_out())

In [7]:
encoded_data.to_csv('../Dataset/CSV-Train/10_1.csv',index_label='Reg_No')